In [1]:
from hail import *
hc = HailContext(tmp_dir = '/nvme/tmp/shusson')

Running on Apache Spark version 2.1.0
SparkUI available at http://129.94.72.55:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.1-c72c76d


In [2]:
vds = hc.read('/nvme/tmp/shusson/mgrb.vds')

In [3]:
vds.summarize()

Summary(samples=0, variants=7432, call_rate=nan, contigs=['X', '12', '8', '19', '4', '15', '11', '9', 'Y', '22', '13', '16', '5', '10', '21', '6', '1', '17', '14', '20', '2', '18', '7', '3'], multiallelics=623, snps=6702, mnps=0, insertions=828, deletions=1008, complex=0, star=339, max_alleles=35)

In [3]:
split = vds.split_multi().annotate_variants_expr('va.allele = va.alleles[va.aIndex-1]')

In [4]:
kt=split.variants_table()

kt = kt.annotate('VARIANT=v, \
CHROMOSOME=v.contig, \
START=v.start, \
REF=v.ref, \
ALT=v.alt, \
RSID=orElse(va.allele.annotation.rsid, "\\\N"), \
AC=va.allele.metrics.allele_counts.total.toInt(), \
AF=if (isMissing(va.allele.metrics.allele_frequencies.total)) "\\\N" else str(va.allele.metrics.allele_frequencies.total),\
TYPE=v.altAllele.category(), \
CATO=if (isMissing(va.allele.annotation.predictions.CATO.score)) "\\\N" else str(va.allele.annotation.predictions.CATO.score), \
eigen=if (isMissing(va.allele.annotation.predictions.Eigen.EigenRaw)) "\\\N" else str(va.allele.annotation.predictions.Eigen.EigenRaw), \
polyPhen=if (isMissing(va.allele.annotation.vep.predictions.PolyPhen)) "\\\N" else str(va.allele.annotation.vep.predictions.PolyPhen), \
sift=if (isMissing(va.allele.annotation.vep.predictions.SIFT)) "\\\N" else str(va.allele.annotation.vep.predictions.SIFT), \
tgpAF=if (isMissing(va.allele.annotation.freqs.tgp.AF)) "\\\N" else str(va.allele.annotation.freqs.tgp.AF), \
hrcAF=if (isMissing(va.allele.annotation.freqs.hrc.AF)) "\\\N" else str(va.allele.annotation.freqs.hrc.AF), \
gnomadAF=if (isMissing(va.allele.annotation.freqs.gnomad.AF)) "\\\N" else str(va.allele.annotation.freqs.gnomad.AF), \
feature=orElse(va.allele.annotation.vep.consequences.feature.id, "\\\N"), \
consequences=orElse(va.allele.annotation.vep.consequences.consequences.mkString(","), "\\\N"), \
gene=orElse(va.allele.annotation.vep.cross_references.Ensembl_gene, "\\\N"), \
clinvar=orElse(va.allele.annotation.clinvar.ClinicalSignificance.mkString(","), "\\\N")')

kt = kt.select([u'VARIANT', u'CHROMOSOME', u'START', u'REF', u'ALT', u'RSID', u'AC', u'AF', u'TYPE', 'CATO', 'eigen', 'sift', 'polyPhen', 'tgpAF', 'hrcAF', 'gnomadAF', 'feature', 'consequences', 'gene', 'clinvar'])


In [5]:
kt.export("/nvme/tmp/shusson/mgrb_mapd.tsv")

2017-10-11 12:20:16 Hail: INFO: while writing:
    /nvme/tmp/shusson/mgrb_mapd.tsv
  merge time: 21.145s


In [ ]:
kt.show()